In [61]:
query = f'''
WITH rider_data AS (SELECT city_id              AS city_id,
                           timezone             AS timezone,
                           type_name            AS type_name,
                           uuid                 AS order_uuid,
                           CAST(NULL AS STRING) AS tender_uuid,
                           'rider_price'        AS price_type,
                           payment_price_value  AS price,
                           price_highrate_value AS price_highrate_value,
                           accepted_tender_uuid AS accepted_tender_uuid,
                           AtoB_seconds         AS AtoB_seconds,
                           CAST(NULL AS INT64)  AS eta,
                           modified_at          AS modified_at_utc,
                           order_done           AS order_done,
                           multiplier           AS multiplier
                    FROM `indriver-e6e40.ods_new_order_rh_cdc.order_global_strm` t1
                             LEFT JOIN (SELECT order_uuid,
                                               MAX(timezone)                                                            AS timezone,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN tender_uuid END)     AS accepted_tender_uuid,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN true ELSE false END) AS order_done,
                                               MAX(duration_in_seconds)                                                 AS AtoB_seconds
                                        FROM `indriver-e6e40.emart.incity_detail`
                                        WHERE true
                                          AND order_uuid IS NOT NULL
                                          AND created_date_order_part BETWEEN
                                            DATE_SUB('2025-01-01', INTERVAL 1 DAY)
                                            AND DATE_ADD('2025-01-21', INTERVAL 1 DAY)
                                        GROUP BY order_uuid) t2
                                       ON t1.uuid = t2.order_uuid
                    WHERE true
                      AND order_uuid IS NOT NULL
                      AND status = 'ORDER_STATUS_ACTIVE'
                      AND AtoB_seconds > 0
                      AND DATE(created_at) BETWEEN
                        DATE_SUB('2025-01-01', INTERVAL 1 DAY)
                        AND DATE_ADD('2025-01-21', INTERVAL 1 DAY)
                    QUALIFY ROW_NUMBER() OVER (PARTITION BY uuid, payment_price_value ORDER BY modified_at) = 1),

     bid_data AS (SELECT DISTINCT CAST(NULL AS INT64)                                      AS city_id,
                                  CAST(NULL AS STRING)                                     AS timezone,
                                  CAST(NULL AS STRING)                                     AS type_name,
                                  order_uuid                                               AS order_uuid,
                                  uuid                                                     AS tender_uuid,
                                  'bid_price'                                              AS price_type,
                                  price                                                    AS price,
                                  CAST(NULL AS INT64)                                      AS price_highrate_value,
                                  CAST(NULL AS STRING)                                     AS accepted_tender_uuid,
                                  CAST(NULL AS INT64)                                      AS AtoB_seconds,
                                  SAFE_CAST(SUBSTR(eta, 1, STRPOS(eta, 's') - 1) AS INT64) AS eta,
                                  modified_at                                              AS modified_at_utc,
                                  CAST(NULL AS BOOL)                                       AS order_done,
                                  CAST(NULL AS INT64)                                      AS multiplier
                  FROM `indriver-e6e40.ods_new_order_rh_cdc.bid_global_strm`
                  WHERE true
                    AND order_uuid IS NOT NULL
                    AND uuid IS NOT NULL
                    AND order_uuid IN (SELECT order_uuid FROM rider_data)
                    AND status = 'BID_STATUS_ACTIVE'
                    AND DATE(created_at) BETWEEN
                      DATE_SUB('2025-01-01', INTERVAL 1 DAY)
                      AND DATE_ADD('2025-01-21', INTERVAL 1 DAY)),

     my_strm AS (SELECT 
                        -- city_id,
                        timezone,
                        -- type_name,
                        order_uuid,
                        tender_uuid,
                        price_type,
                        price,
                        price_highrate_value,
                        accepted_tender_uuid,
                        AtoB_seconds,
                        eta,
                        modified_at_utc,
                        order_done,
                        -- multiplier
                 FROM (SELECT *
                       FROM rider_data
                       UNION ALL
                       SELECT *
                       FROM bid_data)
                 ORDER BY order_uuid, modified_at_utc),

     my_strm_full AS (SELECT 
                            --  LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN city_id END IGNORE NULLS)
                            --             OVER w        AS city_id,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN timezone END IGNORE NULLS)
                                        OVER w        AS timezone,
                            --  LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN type_name END IGNORE NULLS)
                            --             OVER w        AS type_name,
                             order_uuid,
                            tender_uuid,
                             price_type,
                             price,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN price_highrate_value END IGNORE
                                        NULLS) OVER w AS price_highrate_value,
                            LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN accepted_tender_uuid END IGNORE
                                       NULLS) OVER w AS accepted_tender_uuid,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN AtoB_seconds END IGNORE
                                        NULLS) OVER w AS AtoB_seconds,
                             eta,
                             modified_at_utc,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN order_done END IGNORE
                                        NULLS) OVER w AS order_done,

                            --  LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN multiplier END IGNORE
                            --             NULLS) OVER w AS multiplier
                      FROM my_strm
                      WINDOW w AS (
                              PARTITION BY order_uuid
                              ORDER BY modified_at_utc
                              ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING )),

     options_assign AS (SELECT 
                              --  city_id,
                              --  type_name,
                               order_uuid,
                              tender_uuid,
                               price_type,
                              --  price,
                              accepted_tender_uuid,
                               order_done,
                               DATETIME(TIMESTAMP(t1.modified_at_utc), t1.timezone)                      AS modified_at_local,
                               SAFE_DIVIDE(SAFE_DIVIDE(price, (AtoB_seconds + eta)),
                                           SAFE_DIVIDE(price_highrate_value, (AtoB_seconds + 0))) AS ratio,

--                                price / coalesce(multiplier, 100) / t3.usd_value                          AS price_usd
                        FROM my_strm_full AS t1
--                                  LEFT JOIN `indriver-e6e40.ods_monolith.tbl_city` AS t2
--                                            ON t1.city_id = t2.id
--                                  LEFT JOIN `indriver-bi.heap.currency_by_date` AS t3
--                                            ON t2.country_id = t3.country_id
--                                                AND
--                                               DATE(DATETIME(TIMESTAMP(t1.modified_at_utc), t1.timezone)) = t3.date
                        ),

     binned AS (SELECT ratio_bin_start,
                       SAFE_DIVIDE(frequency, SUM(frequency) OVER ()) AS normalized_frequency,
                       SAFE_DIVIDE(bids_accepted_cnt, bids_cnt) AS bid2accept,
                       bids_accepted_cnt, bids_cnt 
                FROM (SELECT count(*)                                    AS frequency,
                             CAST(FLOOR(ratio / 0.05) * 0.05 AS FLOAT64) AS ratio_bin_start,
                             COUNT(price_type) AS bids_cnt,
                             COUNT(IF(tender_uuid = accepted_tender_uuid, price_type, NULL)) AS bids_accepted_cnt
                      FROM options_assign
                      WHERE true
                        AND price_type = 'bid_price'
                        AND DATE(modified_at_local) BETWEEN DATE('2025-01-01') AND DATE('2025-01-21')
                      GROUP BY ratio_bin_start))

SELECT ratio_bin_start,
       normalized_frequency,
       bid2accept,
       bids_accepted_cnt, bids_cnt
--        SUM(normalized_frequency) OVER (PARTITION BY accepted_status ORDER BY bin_start) AS cumulative_frequency
FROM binned
ORDER BY ratio_bin_start;

--
--
'''

In [62]:
query_lima = f'''
WITH rider_data AS (SELECT city_id              AS city_id,
                           timezone             AS timezone,
                           type_name            AS type_name,
                           uuid                 AS order_uuid,
                           CAST(NULL AS STRING) AS tender_uuid,
                           'rider_price'        AS price_type,
                           payment_price_value  AS price,
                           price_highrate_value AS price_highrate_value,
                           accepted_tender_uuid AS accepted_tender_uuid,
                           AtoB_seconds         AS AtoB_seconds,
                           CAST(NULL AS INT64)  AS eta,
                           modified_at          AS modified_at_utc,
                           order_done           AS order_done,
                           multiplier           AS multiplier
                    FROM `indriver-e6e40.ods_new_order_rh_cdc.order_global_strm` t1
                             LEFT JOIN (SELECT order_uuid,
                                               MAX(timezone)                                                            AS timezone,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN tender_uuid END)     AS accepted_tender_uuid,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN true ELSE false END) AS order_done,
                                               MAX(duration_in_seconds)                                                 AS AtoB_seconds
                                        FROM `indriver-e6e40.emart.incity_detail`
                                        WHERE true
                                          AND city_id = 4199
                                          AND order_uuid IS NOT NULL
                                          AND created_date_order_part BETWEEN
                                            DATE_SUB('2025-01-01', INTERVAL 1 DAY)
                                            AND DATE_ADD('2025-01-21', INTERVAL 1 DAY)
                                        GROUP BY order_uuid) t2
                                       ON t1.uuid = t2.order_uuid
                    WHERE true
                      AND city_id = 4199
                      AND order_uuid IS NOT NULL
                      AND status = 'ORDER_STATUS_ACTIVE'
                      AND AtoB_seconds > 0
                      AND DATE(created_at) BETWEEN
                        DATE_SUB('2025-01-01', INTERVAL 1 DAY)
                        AND DATE_ADD('2025-01-21', INTERVAL 1 DAY)
                    QUALIFY ROW_NUMBER() OVER (PARTITION BY uuid, payment_price_value ORDER BY modified_at) = 1),

     bid_data AS (SELECT DISTINCT CAST(NULL AS INT64)                                      AS city_id,
                                  CAST(NULL AS STRING)                                     AS timezone,
                                  CAST(NULL AS STRING)                                     AS type_name,
                                  order_uuid                                               AS order_uuid,
                                  uuid                                                     AS tender_uuid,
                                  'bid_price'                                              AS price_type,
                                  price                                                    AS price,
                                  CAST(NULL AS INT64)                                      AS price_highrate_value,
                                  CAST(NULL AS STRING)                                     AS accepted_tender_uuid,
                                  CAST(NULL AS INT64)                                      AS AtoB_seconds,
                                  SAFE_CAST(SUBSTR(eta, 1, STRPOS(eta, 's') - 1) AS INT64) AS eta,
                                  modified_at                                              AS modified_at_utc,
                                  CAST(NULL AS BOOL)                                       AS order_done,
                                  CAST(NULL AS INT64)                                      AS multiplier
                  FROM `indriver-e6e40.ods_new_order_rh_cdc.bid_global_strm`
                  WHERE true
                    AND order_uuid IS NOT NULL
                    AND uuid IS NOT NULL
                    AND order_uuid IN (SELECT order_uuid FROM rider_data)
                    AND status = 'BID_STATUS_ACTIVE'
                    AND DATE(created_at) BETWEEN
                      DATE_SUB('2025-01-01', INTERVAL 1 DAY)
                      AND DATE_ADD('2025-01-21', INTERVAL 1 DAY)),

     my_strm AS (SELECT 
                        -- city_id,
                        timezone,
                        -- type_name,
                        order_uuid,
                        tender_uuid,
                        price_type,
                        price,
                        price_highrate_value,
                        accepted_tender_uuid,
                        AtoB_seconds,
                        eta,
                        modified_at_utc,
                        order_done,
                        -- multiplier
                 FROM (SELECT *
                       FROM rider_data
                       UNION ALL
                       SELECT *
                       FROM bid_data)
                 ORDER BY order_uuid, modified_at_utc),

     my_strm_full AS (SELECT 
                            --  LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN city_id END IGNORE NULLS)
                            --             OVER w        AS city_id,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN timezone END IGNORE NULLS)
                                        OVER w        AS timezone,
                            --  LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN type_name END IGNORE NULLS)
                            --             OVER w        AS type_name,
                             order_uuid,
                            tender_uuid,
                             price_type,
                             price,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN price_highrate_value END IGNORE
                                        NULLS) OVER w AS price_highrate_value,
                            LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN accepted_tender_uuid END IGNORE
                                       NULLS) OVER w AS accepted_tender_uuid,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN AtoB_seconds END IGNORE
                                        NULLS) OVER w AS AtoB_seconds,
                             eta,
                             modified_at_utc,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN order_done END IGNORE
                                        NULLS) OVER w AS order_done,

                            --  LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN multiplier END IGNORE
                            --             NULLS) OVER w AS multiplier
                      FROM my_strm
                      WINDOW w AS (
                              PARTITION BY order_uuid
                              ORDER BY modified_at_utc
                              ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING )),

     options_assign AS (SELECT 
                              --  city_id,
                              --  type_name,
                               order_uuid,
                              tender_uuid,
                               price_type,
                              --  price,
                              accepted_tender_uuid,
                               order_done,
                               DATETIME(TIMESTAMP(t1.modified_at_utc), t1.timezone)                      AS modified_at_local,
                               SAFE_DIVIDE(SAFE_DIVIDE(price, (AtoB_seconds + eta)),
                                           SAFE_DIVIDE(price_highrate_value, (AtoB_seconds + 0))) AS ratio,

--                                price / coalesce(multiplier, 100) / t3.usd_value                          AS price_usd
                        FROM my_strm_full AS t1
--                                  LEFT JOIN `indriver-e6e40.ods_monolith.tbl_city` AS t2
--                                            ON t1.city_id = t2.id
--                                  LEFT JOIN `indriver-bi.heap.currency_by_date` AS t3
--                                            ON t2.country_id = t3.country_id
--                                                AND
--                                               DATE(DATETIME(TIMESTAMP(t1.modified_at_utc), t1.timezone)) = t3.date
                        ),

     binned AS (SELECT ratio_bin_start,
                       SAFE_DIVIDE(frequency, SUM(frequency) OVER ()) AS normalized_frequency,
                       SAFE_DIVIDE(bids_accepted_cnt, bids_cnt) AS bid2accept,
                       bids_accepted_cnt, bids_cnt 
                FROM (SELECT count(*)                                    AS frequency,
                             CAST(FLOOR(ratio / 0.05) * 0.05 AS FLOAT64) AS ratio_bin_start,
                             COUNT(price_type) AS bids_cnt,
                             COUNT(IF(tender_uuid = accepted_tender_uuid, price_type, NULL)) AS bids_accepted_cnt
                      FROM options_assign
                      WHERE true
                        AND price_type = 'bid_price'
                        AND DATE(modified_at_local) BETWEEN DATE('2025-01-01') AND DATE('2025-01-21')
                      GROUP BY ratio_bin_start))

SELECT ratio_bin_start,
       normalized_frequency,
       bid2accept,
       bids_accepted_cnt, bids_cnt
--        SUM(normalized_frequency) OVER (PARTITION BY accepted_status ORDER BY bin_start) AS cumulative_frequency
FROM binned
ORDER BY ratio_bin_start;

--
--
'''

In [63]:
import os
import re

import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns

from google.cloud.bigquery import Client

client = Client(project='analytics-dev-333113')
df = client.query(query).to_dataframe()

In [64]:
df

,ratio_bin_start,normalized_frequency,bid2accept,bids_accepted_cnt,bids_cnt
0,NaN,3.938141e-02,0.207167,4137730,19972903
1,0.00,5.332103e-04,0.135527,36650,270426
2,0.05,1.894115e-04,0.127614,12259,96063
3,0.10,2.302482e-04,0.117226,13689,116774
4,0.15,3.858995e-04,0.125182,24500,195715
...,...,...,...,...,...
2194,643.10,1.971742e-09,1.000000,1,1
2195,648.60,1.971742e-09,1.000000,1,1
2196,655.85,1.971742e-09,1.000000,1,1
2197,659.20,1.971742e-09,1.000000,1,1


In [121]:
import plotly.graph_objects as go

# Удаляем NaN значения
df = df.dropna(subset=['ratio_bin_start'])

# Создаем график
fig = go.Figure()

# Добавляем гистограмму
fig.add_trace(go.Bar(
    x=df['ratio_bin_start'],
    y=df['normalized_frequency'],
    name='Normalized Frequency',
    marker_color='#66CDAA',  # мятный цвет
    hovertemplate="Ratio: %{x:.2f}<br>Frequency: %{y:.6f}<extra></extra>"
))

# Добавляем линию bid2order на второй оси Y
fig.add_trace(go.Scatter(
    x=df['ratio_bin_start'],
    y=df['bid2accept'],
    name='Bid2Accept',
    marker_color='#FFB4B4',  # розовый цвет
    yaxis='y2',
    hovertemplate="Ratio: %{x:.2f}<br>Bid2Accept: %{y:.3f}<extra></extra>",
    line=dict(width=5)  # Увеличиваем толщину линии
))

# Настраиваем макет
fig.update_layout(
    title='',
    xaxis=dict(
        title='Ratio',
        range=[-0.05, 2.05],
        title_font=dict(size=12),
        tickfont=dict(size=12)
    ),
    yaxis=dict(
        title='Normalized Frequency',
        title_font=dict(size=12),
        tickfont=dict(size=12)
    ),
    yaxis2=dict(
        title='Bid2Order',
        overlaying='y',
        side='right',
        range=[0, 0.3],
        title_font=dict(size=12),
        tickfont=dict(size=12)
    ),
    showlegend=True,
    plot_bgcolor='white',
    legend=dict(
        yanchor="top",
        y=1.0,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.8)",  # полупрозрачный белый фон
        bordercolor="rgba(0, 0, 0, 0.2)",    # рамка
        borderwidth=1,
        font=dict(size=12)  
    ),
    margin=dict(t=100),  # увеличиваем верхний отступ для легенды
    hovermode='x unified',
    width=1000,  
    height=1000*9/16,
    title_font=dict(size=15)  # Увеличиваем размер шрифта заголовка
    # xaxis=dict(title_font=dict(size=12), tickfont=dict(size=14)),  # Увеличиваем размер шрифта оси X
    # yaxis=dict(title_font=dict(size=12), tickfont=dict(size=14)),  # Увеличиваем размер шрифта оси Y
    # yaxis2=dict(title_font=dict(size=16), tickfont=dict(size=14))  # Увеличиваем размер шрифта второй оси Y
)

# # Добавляем сетку
# fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#E5E5E5')
# fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#E5E5E5')

# Показываем график
fig.show()

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [74]:
client = Client(project='analytics-dev-333113')
df_lima = client.query(query_lima).to_dataframe()
df_lima

,ratio_bin_start,normalized_frequency,bid2accept,bids_accepted_cnt,bids_cnt
0,NaN,1.415553e-02,0.158000,101621,643171
1,0.00,1.670921e-04,0.133562,1014,7592
2,0.05,4.003432e-05,0.150632,274,1819
3,0.10,3.365172e-05,0.190974,292,1529
4,0.15,4.692313e-05,0.205441,438,2132
...,...,...,...,...,...
785,150.20,4.401794e-08,0.000000,0,2
786,150.85,2.200897e-08,0.000000,0,1
787,166.15,2.200897e-08,0.000000,0,1
788,199.40,2.200897e-08,0.000000,0,1
